In [ ]:
!pip install pandas scikit-learn metric-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from metric_learn import LMNN
from sklearn.kernel_ridge import KernelRidge

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/parkinsons_updrs.data')

In [ ]:
data.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [ ]:
# Drop the non-predictive 'subject#'
X = data.drop(columns=['subject#', 'motor_UPDRS', 'total_UPDRS'])
y_motor = data['motor_UPDRS']
y_total = data['total_UPDRS']

X.head()

,age,sex,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,72,0,5.6431,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,72,0,12.6660,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,72,0,19.6810,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,72,0,25.6470,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,72,0,33.6420,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [ ]:
# Split into train and test (70% train)
X_train, X_test, y_motor_train, y_motor_test, y_total_train, y_total_test = train_test_split(
      X, y_motor, y_total, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
# Possible M values
p = X.shape[1]
M_values = [5, 10, 15, p]

def run_metric_learning(X_train_std, y_train, X_test_std, y_test, output_name):
    print(f"\n--- {output_name} ---")
    best_r2 = -np.inf
    best_m = None

    for M in M_values:
        # 5-fold cross-validation
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        val_scores = []

        for train_idx, val_idx in cv.split(X_train_std):
            X_tr, X_val = X_train_std[train_idx], X_train_std[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            # PCA for initialization
            pca = PCA(n_components=M)
            X_tr_pca = pca.fit_transform(X_tr)
            X_val_pca = pca.transform(X_val)

            # Metric Learning (can be LMNN or another method)
            # Use Kernel Ridge Regression with Gaussian kernel
            kr = KernelRidge(kernel='rbf', gamma=0.1)
            kr.fit(X_tr_pca, y_tr)
            y_val_pred = kr.predict(X_val_pca)
            val_scores.append(r2_score(y_val, y_val_pred))

        avg_r2 = np.mean(val_scores)
        print(f"M = {M}, CV R2 = {avg_r2:.4f}")

        if avg_r2 > best_r2:
            best_r2 = avg_r2
            best_m = M

    # Train final model with best M
    print(f"Best M = {best_m}")
    pca = PCA(n_components=best_m)
    X_train_final = pca.fit_transform(X_train_std)
    X_test_final = pca.transform(X_test_std)

    model = KernelRidge(kernel='rbf', gamma=0.1)
    model.fit(X_train_final, y_train)

    # Final evaluation
    y_train_pred = model.predict(X_train_final)
    y_test_pred = model.predict(X_test_final)

    print(f"Train R2: {r2_score(y_train, y_train_pred):.4f}")
    print(f"Test R2 : {r2_score(y_test, y_test_pred):.4f}")

# (b) Run for both outputs
run_metric_learning(X_train_std, y_motor_train, X_test_std, y_motor_test, 'motor_UPDRS')
run_metric_learning(X_train_std, y_total_train, X_test_std, y_total_test, 'total_UPDRS')


--- motor_UPDRS ---
M = 5, CV R2 = 0.2878
M = 10, CV R2 = 0.5043
M = 15, CV R2 = 0.5036
M = 19, CV R2 = 0.5036
Best M = 10
Train R2: 0.6023
Test R2 : 0.5125

--- total_UPDRS ---
M = 5, CV R2 = 0.2856
M = 10, CV R2 = 0.4782
M = 15, CV R2 = 0.4803
M = 19, CV R2 = 0.4801
Best M = 15
Train R2: 0.6034
Test R2 : 0.4823


In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
# Define features and labels
y = data[['motor_UPDRS', 'total_UPDRS']]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.7, random_state=42)

# Function to train and evaluate the model
def train_and_evaluate(hidden_layer_sizes, activation, solver, alpha, learning_rate):
    # Define the MLPRegressor model
    mlp = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=2000,
        random_state=42,
        early_stopping=False
    )

    # Train the model
    mlp.fit(X_train, y_train)

    # Predict on training and test sets
    y_train_pred = mlp.predict(X_train)
    y_test_pred = mlp.predict(X_test)

    # Calculate R2 scores
    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    return train_r2, test_r2

# Example parameters (tune these by trial and error)
hidden_layer_sizes = (50,)  # Single hidden layer with 50 neurons
activation = 'relu'
solver = 'adam'
alpha = 0.001  # L2 regularization parameter
learning_rate = 'constant'

train_r2, test_r2 = train_and_evaluate(hidden_layer_sizes, activation, solver, alpha, learning_rate)

print("Training R2:", train_r2)
print("Test R2:", test_r2)

# Loop through different parameter combinations to find the best architecture
results = []
for hidden_layer_sizes in [(50,), (100,), (50, 50)]:
    for activation in ['relu', 'tanh']:
        for solver in ['adam', 'sgd']:
            for alpha in [0.0001, 0.001, 0.01]:
                for learning_rate in ['constant', 'adaptive']:
                    train_r2, test_r2 = train_and_evaluate(hidden_layer_sizes, activation, solver, alpha, learning_rate)
                    results.append({
                        'hidden_layer_sizes': hidden_layer_sizes,
                        'activation': activation,
                        'solver': solver,
                        'alpha': alpha,
                        'learning_rate': learning_rate,
                        'train_r2': train_r2,
                        'test_r2': test_r2
                    })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)


# Display the best parameters based on test R2 score
best_params = results_df.loc[results_df['test_r2'].idxmax()]
print("Best Config:", best_params)

Training R2: 0.7219636893055364
Test R2: 0.6469816831608961


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

Best Config: hidden_layer_sizes    (50, 50)
activation                tanh
solver                     sgd
alpha                     0.01
learning_rate         constant
train_r2              0.976593
test_r2               0.850079
Name: 70, dtype: object


In [ ]:
def train_and_evaluate_with_early_stopping(hidden_layer_sizes, activation, solver, alpha, learning_rate, validation_fraction):
    # Define the MLPRegressor model
    mlp = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=2000,
        random_state=42,
        early_stopping=True,
        validation_fraction=validation_fraction
    )

    # Train the model
    mlp.fit(X_train, y_train)

    # Predict on training and test sets
    y_train_pred = mlp.predict(X_train)
    y_test_pred = mlp.predict(X_test)

    # Calculate R2 scores
    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    return train_r2, test_r2

# Example parameters based on the best parameters from the previous part
hidden_layer_sizes = (50,50)  # Single hidden layer with 50 neurons
activation = 'tanh'
solver = 'sgd'
alpha = 0.01  # L2 regularization parameter
learning_rate = 'constant'
validation_fraction = 0.1  # Default validation fraction

train_r2, test_r2 = train_and_evaluate_with_early_stopping(hidden_layer_sizes, activation, solver, alpha, learning_rate, validation_fraction)

print("Training R2:", train_r2)
print("Test R2:", test_r2)

# Experiment with different validation fractions to see if a better model can be obtained
results = []
for validation_fraction in [0.1, 0.2, 0.3]:
    train_r2, test_r2 = train_and_evaluate_with_early_stopping(hidden_layer_sizes, activation, solver, alpha, learning_rate, validation_fraction)
    results.append({
        'validation_fraction': validation_fraction,
        'train_r2': train_r2,
        'test_r2': test_r2
    })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)


# Display the best parameters based on test R2 score
best_params = results_df.loc[results_df['test_r2'].idxmax()]
print("Best Config:", best_params)

Training R2: 0.802086156122523
Test R2: 0.7329439538341394
Best Config: validation_fraction    0.300000
train_r2               0.911098
test_r2                0.823081
Name: 2, dtype: float64
